In [1]:
from google.colab import drive
drive.mount('/content/drive')




ModuleNotFoundError: No module named 'google.colab'

In [9]:
!conda install tensorflow scikit-image jupyter matplotlib numpy pandas


Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: win-64


Out of memory allocating 30408704 bytes!


In [7]:
import numpy as np
import rasterio
from rasterio.features import rasterize
import geopandas as gpd
from osgeo import gdal
import fiona


ModuleNotFoundError: No module named 'rasterio'

In [4]:
# Step 3: Define file paths
tiff_path = "/content/drive/My Drive/Chhattisgarh_2/ecw/chattissgarh.tif"
gdb_path = "/content/drive/My Drive/Chhattisgarh_2/Chhattisgarh_Svamitva.gdb"



In [5]:
# Step 4: Load raster data
tiff_dataset = gdal.Open(tiff_path)
if tiff_dataset is None:
    print("Failed to load TIFF file!")
else:
    print("TIFF file loaded successfully")

TIFF file loaded successfully


In [6]:
# Step 5: Get raster band and size
raster_band = tiff_dataset.GetRasterBand(1)
raster_data = raster_band.ReadAsArray()
height, width = raster_data.shape


In [7]:
chunk_size = 1000  # Define your chunk size
width = tiff_dataset.RasterXSize
height = tiff_dataset.RasterYSize

In [8]:
layers = fiona.listlayers(gdb_path)
print("Available Layers:", layers)

Available Layers: ['Built_Up_Area_type', 'Water_Body', 'Waterbody_Point', 'Railway', 'Bridge', 'Utility', 'Road_Centre_Line', 'Road', 'Water_Body_Line', 'Road_line']


In [9]:
# Step 7: Read relevant vector layers
built_up_area_features = gpd.read_file(gdb_path, layer='Built_Up_Area_type')
water_features = gpd.read_file(gdb_path, layer='Water_Body')
road_features = gpd.read_file(gdb_path, layer='Road_Centre_Line')


In [10]:
# Step 8: Check for empty geometries and filter them out
def filter_empty_geometries(gdf):
    """Filter out geometries that are invalid or empty."""
    valid_geometries = gdf[gdf.is_valid & ~gdf.is_empty]
    return valid_geometries

In [11]:
built_up_area_features = filter_empty_geometries(built_up_area_features)
water_features = filter_empty_geometries(water_features)
road_features = filter_empty_geometries(road_features)


In [12]:
# Step 9: Ensure layers loaded correctly
print(built_up_area_features.head(), end='\n\n')
print(water_features.head(), end='\n\n')
print(road_features.head(), end='\n\n')


  GlobalID Uniq_Id State_Code District_Code Tehsil_Code Block_Code  \
0     None                 22           719         719       3629   
1     None                 22           719         719       3629   
2     None                 22           719         719       3629   
3     None                 22           719         719       3629   
4     None                 22           719         719       3629   

  Village_Code LGD_Code  Built_Up_Area_type Village_Name  ...    Area_Sqm  \
0       443193   443193                   1        AMORA  ...   10.126829   
1       443193   443193                   1        AMORA  ...   10.691340   
2       443193   443193                   1        AMORA  ...    8.892256   
3       443193   443193                   1        AMORA  ...  142.584709   
4       443193   443193                   1        AMORA  ...    4.850011   

  Roof_type No_Floors Remarks Adl_Info  SHAPE_Length  SHAPE_Area  GP_Code  \
0       2.0       1.0                  

In [13]:
# Step 10: Define rasterization function to convert geometries into raster form
def rasterize_features(features, out_shape, transform, label_value):
    """Rasterizes a given GeoDataFrame's geometries to a label value."""
    geometries = [(geom, label_value) for geom in features.geometry]
    rasterized = rasterize(
        geometries,
        out_shape=out_shape,
        transform=transform,
        fill=0,  # Background
        dtype=np.uint8
    )
    return rasterized


In [15]:
transform = rasterio.transform.from_bounds(*tiff_dataset.GetGeoTransform(), width, height)

TypeError: from_bounds() takes 6 positional arguments but 8 were given

In [14]:
# Step 11: Rasterize the features into different labels

transform = rasterio.transform.from_bounds(*tiff_dataset.GetGeoTransform()[:4], width, height)

# Roads (label=1), Water bodies (label=2), Built-up areas (label=3)
roads_raster = rasterize_features(road_features, (height, width), transform, label_value=1)
water_raster = rasterize_features(water_features, (height, width), transform, label_value=2)
built_up_raster = rasterize_features(built_up_area_features, (height, width), transform, label_value=3)

In [15]:
y = np.zeros((height, width), dtype=np.uint8)
y[roads_raster == 1] = 1
y[water_raster == 2] = 2
y[built_up_raster == 3] = 3


In [16]:
X = raster_data.reshape((height, width, 1))


In [17]:
print("Rasterized label shape:", y.shape)
print("Sample values from combined labels:", np.unique(y))

Rasterized label shape: (12555, 16609)
Sample values from combined labels: [0]


# MODEL BUILDING STARTS HERE

In [18]:
import numpy as np

# Reshape `X` to have 3 dimensions (height, width, channels)
height, width = raster_data.shape  # Assuming `raster_data` is 2D (grayscale)
X = raster_data.reshape((height, width, 1))  # Reshaping to (height, width, 1) for grayscale

# `y` should already be a 2D array where each pixel has its label (0, 1, 2, 3)
# Ensure that the shape of `y` is (height, width)
assert y.shape == (height, width), "Label shape mismatch!"


In [34]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [1]:
# Import necessary libraries
import numpy as np
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Resize the input data
height_resized, width_resized = 256, 256

# Resize the image data
X_resized = np.array([resize(image, (height_resized, width_resized, 1), preserve_range=True, anti_aliasing=True) for image in X])

# No need to resize y unless your task requires it, but you may convert it to categorical if needed
y_resized = y

model = Sequential()

model.add(Input(shape=(height_resized, width_resized, 1)))
model.add(Conv2D(16, (3, 3), activation='relu'))  # Reduce number of filters
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))  # Reduce the number of neurons
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Now split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resized, y_resized, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

NameError: name 'X' is not defined